# 使用自動化機器學習來定型分類模型

有許多種類的機器學習演算法可用來定型模型，有時不容易判斷特定資料和預測需求的最有效演算法。 此外，您可以使用正規化、遺漏特徵插補等技術，預先處理定型資料，大幅影響模型的預測效能。 在尋求尋找您需求的最佳模型時，您可能需要嘗試許多演算法和前置處理轉換的組合;這需要大量時間和計算資源。

Azure Machine Learning 可讓您自動比較使用不同演算法和前置處理選項定型的模型。 您可以使用 [Azure Machine Learning Studio](https://ml/azure.com) 中的視覺化介面或 Python SDK (v2) 來運用這項功能。 Python SDK 可讓您更充分地控制自動化機器學習作業的設定，但視覺化介面更容易使用。

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 準備資料

您不需要建立自動化機器學習的定型腳本，但您需要載入定型資料。 

在此情況下，您將使用包含糖尿病病患詳細資料的資料集。 

若要將資料集當做輸入傳遞至自動化機器學習作業，資料必須採用表格式格式，並包含目標資料行。 若要將資料解譯為表格式資料集，輸入資料集必須是 **MLTable**。

MLTable 資料資產已在設定期間為您建立。 您可以流覽至 [資料] 頁面來探索 **資料** 資產。 您可以藉由指定其名稱和 `diabetes-training-table` 版本 `1` ，在這裡擷取資料資產。 

In [ ]:
## 設定自動化機器學習作業

現在，您已準備好設定自動化機器學習實驗。

當您執行下列程式碼時，它會建立自動化機器學習作業：.

- 使用名為 的計算叢集 `aml-cluster`
- 將 設定 `Diabetic` 為目標資料行
- 將 設定 `accuracy` 為主要計量
- 在訓練時間總計幾分鐘後 `60` 逾時 
- 定型模型 `5` 數目上限
- 不會使用 `LogisticRegression` 演算法來定型模型

In [ ]:
## 執行自動化機器學習作業

確定，您已準備好繼續。 讓我們執行自動化機器學習實驗。

> **注意**：這可能需要一些時間！

當作業正在執行時，您可以在 Studio 中監視它。

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.